In [ ]:
'''
input: .svg vectors

output: gcodes - relative
    cur y = y1: 
        cur x = x1 => go to x2 with power
        cur x != x1 => go to x1, go to x2 with power
    next y:
        go to x1 y1, do (cur y = y1)
'''

'''
Params: size_scale, power_scale, offset (dx, dy)

Gcode config:
    G0 X0 Y0: move fast to 0, 0 
    G1 X0 Y0 F1000: move to 0, 0 with feed_rate 1000 mm/min
    M3 S255: turn laser on at max power
    M5: turn laser off

    G21: metric (mm unit)
    G90: absolute position mode
'''

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def svg2gcode(size_scale=1.0, power_scale=1.0, dx=0, dy=0):
    svg_file = "images/test2.svg"
    tree = ET.parse(svg_file)
    root = tree.getroot()

    max_laser_power = 255
    feed_rate = 500

    gcode = ["G21", "G90", f"F{feed_rate}"]
    cur_x, cur_y, cur_laser = dx, dy, 0

    for line in root.findall(".//{http://www.w3.org/2000/svg}line"):
        x1 = float(line.attrib['x1']) * size_scale + dx
        y1 = float(line.attrib['y1']) * size_scale + dy
        x2 = float(line.attrib['x2']) * size_scale + dx
        opacity = float(line.attrib.get('stroke-opacity', "1.0"))
        laser_power = int(opacity * max_laser_power * power_scale)

        if (cur_x, cur_y) != (x1, y1):
            if cur_x != x1:
                gcode.append(f"G0 X{x1} Y{y1}")
            cur_x, cur_y = x1, y1

        if cur_laser != laser_power:
            gcode.append(f"M3 S{laser_power}")
            cur_laser = laser_power

        gcode.append(f"G1 X{x2}")
        cur_x = x2

    gcode.append("M5")
    gcode.append(f"G0 X{dx} Y{dy}")

    return "\n".join(gcode)


In [ ]:
gcode_output = svg2gcode()
with open("test2.gcode", "w") as file:
    file.write(gcode_output)